# Community Modeling with PyCoMo
PyCoMo is a python package that allows for the modeling and simulation of microbial communities, focusing on their interactions, metabolism, and dynamics. PyCoMo provides a set of tools to simulate and analyze how different species in a community interact with each other, share resources, and contribute to the overall ecosystem's function. Here, we will use the representative organisms identified from the t-SNE clustering to build the community model.

In [1]:
# Installing necessary packages and importing libraries
!pip install pycomo
!pip install cobra
import pycomo
import pandas as pd
import os
import cobra
from cobra.io import read_sbml_model
import matplotlib.pyplot as plt
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 39.4 MB/s eta 0:00:00


2024-12-04 18:12:32,771 - pycomo - INFO - Logger initialized.
INFO:pycomo:Logger initialized.
2024-12-04 18:12:37,709 - pycomo - INFO - Utils Logger initialized.
INFO:pycomo:Utils Logger initialized.
2024-12-04 18:12:37,715 - pycomo - INFO - Multiprocess Logger initialized.
INFO:pycomo:Multiprocess Logger initialized.


# Creating a community model
The creation of a community model consists of 3 steps:
1. Loading the member models
2. Preparing the member models for merging
3. Creating a community model

In [2]:
# ### Loading the member models ###

from google.colab import drive
drive.mount('/content/drive')

# Load the CSV file
df = pd.read_csv("/content/drive/My Drive/AGORA/CurrentVersion/AGORA_1_03/AGORA_1_03_sbml/representative_organisms.csv")
representative_organisms = df["Organism"].tolist()  # Convert back to a list
print(representative_organisms)

Mounted at /content/drive
['Streptococcus_anginosus_1_2_62CV', 'Bacteroides_fragilis_NCTC_9343', 'Streptococcus_thoraltensis_DSM_12221', 'Shigella_sonnei_Ss046', 'Sulfolobus_solfataricus_P2', 'Porphyromonas_uenonis_60_3', 'Clostridium_bartlettii_DSM_16795']


In [6]:
# function to store the organism name and the model path as a dictionary
def import_models(file_path, extension='.xml'):
  models={}
  for organism_id in representative_organisms:
    file_name = organism_id
    models[file_name] = file_path + organism_id + extension

  return models

file_path = '/content/drive/My Drive/AGORA/CurrentVersion/AGORA_1_03/AGORA_1_03_sbml/'
models=import_models(file_path)
models

{'Streptococcus_anginosus_1_2_62CV': '/content/drive/My Drive/AGORA/CurrentVersion/AGORA_1_03/AGORA_1_03_sbml/Streptococcus_anginosus_1_2_62CV.xml',
 'Bacteroides_fragilis_NCTC_9343': '/content/drive/My Drive/AGORA/CurrentVersion/AGORA_1_03/AGORA_1_03_sbml/Bacteroides_fragilis_NCTC_9343.xml',
 'Streptococcus_thoraltensis_DSM_12221': '/content/drive/My Drive/AGORA/CurrentVersion/AGORA_1_03/AGORA_1_03_sbml/Streptococcus_thoraltensis_DSM_12221.xml',
 'Shigella_sonnei_Ss046': '/content/drive/My Drive/AGORA/CurrentVersion/AGORA_1_03/AGORA_1_03_sbml/Shigella_sonnei_Ss046.xml',
 'Sulfolobus_solfataricus_P2': '/content/drive/My Drive/AGORA/CurrentVersion/AGORA_1_03/AGORA_1_03_sbml/Sulfolobus_solfataricus_P2.xml',
 'Porphyromonas_uenonis_60_3': '/content/drive/My Drive/AGORA/CurrentVersion/AGORA_1_03/AGORA_1_03_sbml/Porphyromonas_uenonis_60_3.xml',
 'Clostridium_bartlettii_DSM_16795': '/content/drive/My Drive/AGORA/CurrentVersion/AGORA_1_03/AGORA_1_03_sbml/Clostridium_bartlettii_DSM_16795.xml'}

In [8]:
# Loading the models with inbuilt load_named_model function and storing them as a dictionary
loaded_named_models = []
for model in models.keys():
  named_models = pycomo.load_named_model(file_path + model + ".xml")
  loaded_named_models.append(named_models)

named_models = dict(loaded_named_models)
named_models

{<Model M_Streptococcus_anginosus_1_2_62CV__44____32__AGORA__32__version__32__1__46__03 at 0x79ceccc074f0>: 'Streptococcus_anginosus_1_2_62CV',
 <Model M_Bacteroides_fragilis_NCTC_9343__44____32__AGORA__32__version__32__1__46__03 at 0x79cecd0e0b80>: 'Bacteroides_fragilis_NCTC_9343',
 <Model M_Streptococcus_thoraltensis_DSM_12221__44____32__AGORA__32__version__32__1__46__03 at 0x79cecc241660>: 'Streptococcus_thoraltensis_DSM_12221',
 <Model M_Shigella_sonnei_Ss046__44____32__AGORA__32__version__32__1__46__03 at 0x79cecc999a20>: 'Shigella_sonnei_Ss046',
 <Model M_Sulfolobus_solfataricus_P2__44____32__AGORA__32__version__32__1__46__03 at 0x79cecb7613c0>: 'Sulfolobus_solfataricus_P2',
 <Model M_Porphyromonas_uenonis_60_3__44____32__AGORA__32__version__32__1__46__03 at 0x79cec9b65720>: 'Porphyromonas_uenonis_60_3',
 <Model M_Clostridium_bartlettii_DSM_16795__44____32__AGORA__32__version__32__1__46__03 at 0x79cec9928f70>: 'Clostridium_bartlettii_DSM_16795'}

<p align="justify">

One of the requirements for a community metabolic model is a common biomass function.Let's check if the biomass function is the objective in all the models.</p>

**Investigating the objective function**

In [9]:
for model in named_models.keys():
    print(model.objective)

Maximize
1.0*biomass528 - 1.0*biomass528_reverse_17df4
Maximize
1.0*biomass345 - 1.0*biomass345_reverse_e128f
Maximize
1.0*biomass109 - 1.0*biomass109_reverse_00b6a
Maximize
1.0*biomass355 - 1.0*biomass355_reverse_0f45b
Maximize
1.0*biomass249 - 1.0*biomass249_reverse_b6f37
Maximize
1.0*biomass368 - 1.0*biomass368_reverse_fafd3
Maximize
1.0*biomass524 - 1.0*biomass524_reverse_2f2dd


<p align="justify">According of above results, we can confirm that all the model organisms have the biomass reaction as the objective expression.

With the models loaded, the next step is preparing them for merging. This is done by creating SingleOrganismModel objects. Using them, the models will be formatted for compliance with the SBML format. Further, an exchange compartment will be generated under the name _medium_.</p>

In [10]:
single_org_models = []
for model, name in named_models.items():
    print(name)
    single_org_model = pycomo.SingleOrganismModel(model, name)
    single_org_models.append(single_org_model)

Streptococcus_anginosus_1_2_62CV
Bacteroides_fragilis_NCTC_9343
Streptococcus_thoraltensis_DSM_12221
Shigella_sonnei_Ss046
Sulfolobus_solfataricus_P2
Porphyromonas_uenonis_60_3
Clostridium_bartlettii_DSM_16795


**Creating a community model**

<p align="justify">With the member models prepared, the community model can be generated. The first step is to create a CommunityModel objects from the member models.</p>

In [11]:
# ### Creating a community model ###
community_name = "Representative_Gut_Microbiome"
com_model_obj = pycomo.CommunityModel(single_org_models, community_name)
com_model_obj.model

2024-12-04 18:42:57,496 - pycomo - INFO - No community model generated yet. Generating now:
INFO:pycomo:No community model generated yet. Generating now:
2024-12-04 18:42:57,663 - pycomo - INFO - Note: no products in the objective function, adding biomass to it.
INFO:pycomo:Note: no products in the objective function, adding biomass to it.
2024-12-04 18:43:04,072 - pycomo - INFO - Note: no products in the objective function, adding biomass to it.
INFO:pycomo:Note: no products in the objective function, adding biomass to it.
2024-12-04 18:43:08,236 - pycomo - WARNING - WARNING: no annotation overlap found for matching metabolite pb. Please make sure that the metabolite with this ID is indeed representing the same substance in all models!
2024-12-04 18:43:08,243 - pycomo - WARNING - WARNING: no annotation overlap found for matching metabolite hg2. Please make sure that the metabolite with this ID is indeed representing the same substance in all models!
2024-12-04 18:43:08,250 - pycomo - 

Name,Representative_Gut_Microbiome
Memory address,79ced1bb5540
Number of metabolites,19598
Number of reactions,20696
Number of genes,4971
Number of groups,67
Objective expression,1.0*community_biomass - 1.0*community_biomass_reverse_44dc1
Compartments,"Streptococcus_anginosus_1_2_62CV_c, Streptococcus_anginosus_1_2_62CV_e, Streptococcus_anginosus_1_2_62CV_medium, medium, fraction_reaction, Bacteroides_fragilis_NCTC_9343_c, Bacteroides_fragilis_NCTC_9343_e, Bacteroides_fragilis_NCTC_9343_medium, Streptococcus_thoraltensis_DSM_12221_c, Streptococcus_thoraltensis_DSM_12221_e, Streptococcus_thoraltensis_DSM_12221_medium, Shigella_sonnei_Ss046_c, Shigella_sonnei_Ss046_e, Shigella_sonnei_Ss046_medium, Sulfolobus_solfataricus_P2_e, Sulfolobus_solfataricus_P2_c, Sulfolobus_solfataricus_P2_medium, Porphyromonas_uenonis_60_3_c, Porphyromonas_uenonis_60_3_e, Porphyromonas_uenonis_60_3_medium, Clostridium_bartlettii_DSM_16795_c, Clostridium_bartlettii_DSM_16795_e, Clostridium_bartlettii_DSM_16795_medium"


The community model object has two utility methods to display information on the model.

1) Summary - behaves the same as the summary method of COBRApy, displaying the the solution of FBA and its exchange metabolites

In [12]:
com_model_obj.summary()


Metabolite,Reaction,Flux,C-Number,C-Flux
arg_L_medium,EX_arg_L_medium,0.2883,6,0.93%
asn_L_medium,EX_asn_L_medium,0.2347,4,0.50%
asp_L_medium,EX_asp_L_medium,19.77,4,42.48%
ca2_medium,EX_ca2_medium,0.007809,0,0.00%
cgly_medium,EX_cgly_medium,0.1046,5,0.28%
cl_medium,EX_cl_medium,0.007809,0,0.00%
cobalt2_medium,EX_cobalt2_medium,0.007809,0,0.00%
cu2_medium,EX_cu2_medium,0.007809,0,0.00%
fe2_medium,EX_fe2_medium,0.01562,0,0.00%
fru_medium,EX_fru_medium,7.327,6,23.62%


2) Report

The report function displays information on the model structure: the number of metabolites, reactions, genes, etc., but also quality control measures on mass and charge balance and internal loops.

In [13]:
com_model_obj.report()

2024-12-04 18:56:51,714 - pycomo - INFO - Note: The model has more than 5000 reactions. Calculation of loops is skipped, as this would take some time. If needed, please run manually via .get_loops()
INFO:pycomo:Note: The model has more than 5000 reactions. Calculation of loops is skipped, as this would take some time. If needed, please run manually via .get_loops()


Name: Representative_Gut_Microbiome
------------------
Model overview
Model structure: fixed growth rate
# Metabolites: 19598
# Constraint (f-) Metabolites: 12020
# Model Metabolites: 7578
# Reactions: 20696
# Constraint (f-) Reactions: 12019
# Model Reactions: 8677
# Genes: 4971
# Members: 7
Members:
	Streptococcus_anginosus_1_2_62CV
	Bacteroides_fragilis_NCTC_9343
	Streptococcus_thoraltensis_DSM_12221
	Shigella_sonnei_Ss046
	Sulfolobus_solfataricus_P2
	Porphyromonas_uenonis_60_3
	Clostridium_bartlettii_DSM_16795
Objective in direction max:
	1.0*community_biomass - 1.0*community_biomass_reverse_44dc1
------------------
Model quality
# Reactions unbalanced: 87
# Reactions able to carry flux without a medium: NaN


{'community_name': 'Representative_Gut_Microbiome',
 'model_structure': 'fixed growth rate',
 'num_metabolites': 19598,
 'num_f_metabolites': 12020,
 'num_model_metabolites': 7578,
 'num_reactions': 20696,
 'num_f_reactions': 12019,
 'num_model_reactions': 8677,
 'num_genes': 4971,
 'member_names': ['Streptococcus_anginosus_1_2_62CV',
  'Bacteroides_fragilis_NCTC_9343',
  'Streptococcus_thoraltensis_DSM_12221',
  'Shigella_sonnei_Ss046',
  'Sulfolobus_solfataricus_P2',
  'Porphyromonas_uenonis_60_3',
  'Clostridium_bartlettii_DSM_16795'],
 'num_members': 7,
 'objective_expression': 1.0*community_biomass - 1.0*community_biomass_reverse_44dc1,
 'objective_direction': 'max',
 'unbalanced_reactions': {<Reaction Streptococcus_anginosus_1_2_62CV_DM_2HYMEPH_Streptococcus_anginosus_1_2_62CV_c at 0x79cec5373640>: {'C': -7.0,
   'H': -8.0,
   'O': -2.0},
  <Reaction Streptococcus_anginosus_1_2_62CV_DM_HQN_Streptococcus_anginosus_1_2_62CV_c at 0x79cec5373700>: {'C': -6.0,
   'H': -6.0,
   'O': -2

<p align="justify">By default the community model object will have the structure of fixe growth rate. This means, the fractions of the community member abundance is allowed to vary during simulations, but the individual and community growth rate is set to a fixed value (default: 1.0). To visualize the potential cross-feeding relationships between orhanisms, a fixed equal abundance community was used.</p>


In [14]:
com_model_obj.convert_to_fixed_abundance()
abundance_dict = com_model_obj.generate_equal_abundance_dict()
com_model_obj.apply_fixed_abundance(abundance_dict)
com_model_obj.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
_26dap_M_medium,EX__26dap_M_medium,0.7613,7,0.06%
_2dmmq8_medium,EX__2dmmq8_medium,0.1154,50,0.06%
_2obut_medium,EX__2obut_medium,135.1,4,5.67%
_3mop_medium,EX__3mop_medium,10.9,6,0.69%
_4abz_medium,EX__4abz_medium,0.1517,7,0.01%
_4hbz_medium,EX__4hbz_medium,0.02529,7,0.00%
acgam_medium,EX_acgam_medium,1.634,8,0.14%
acnam_medium,EX_acnam_medium,0.817,11,0.09%
adn_medium,EX_adn_medium,3.264,10,0.34%
akg_medium,EX_akg_medium,4.212,5,0.22%


In [15]:
com_model_obj.model.optimize()

,fluxes,reduced_costs
Streptococcus_anginosus_1_2_62CV__2HMCOXT_Streptococcus_anginosus_1_2_62CV_c,0.000000,-0.000000e+00
Streptococcus_anginosus_1_2_62CV__2MBCOATA_Streptococcus_anginosus_1_2_62CV_c,0.000000,-0.000000e+00
Streptococcus_anginosus_1_2_62CV__3HAD100_Streptococcus_anginosus_1_2_62CV_c,0.000000,0.000000e+00
Streptococcus_anginosus_1_2_62CV__3HAD10M11_Streptococcus_anginosus_1_2_62CV_c,0.000000,0.000000e+00
Streptococcus_anginosus_1_2_62CV__3HAD10M12_Streptococcus_anginosus_1_2_62CV_c,0.000000,0.000000e+00
...,...,...
SK_Clostridium_bartlettii_DSM_16795_TF_no3_Clostridium_bartlettii_DSM_16795_e_ub,1.428571,0.000000e+00
SK_Clostridium_bartlettii_DSM_16795_to_community_biomass_ub,1.346908,0.000000e+00
f_final,1.000000,5.120094e-13
abundance_reaction,57.164136,2.220446e-16


In [16]:
abundance_dict # Investigating the abundances of each organism

{'Streptococcus_anginosus_1_2_62CV': 0.14285714285714285,
 'Bacteroides_fragilis_NCTC_9343': 0.14285714285714285,
 'Streptococcus_thoraltensis_DSM_12221': 0.14285714285714285,
 'Shigella_sonnei_Ss046': 0.14285714285714285,
 'Sulfolobus_solfataricus_P2': 0.14285714285714285,
 'Porphyromonas_uenonis_60_3': 0.14285714285714285,
 'Clostridium_bartlettii_DSM_16795': 0.14285714285714285}

In [17]:
com_model_obj.save("/content/drive/My Drive/Rotation 1/Representative_Gut_Microbiome.xml")

In [18]:
com_model_obj.fba_solution_flux_vector('/content/drive/My Drive/Rotation 1/FBA_flux_file')

2024-12-04 19:37:04,162 - pycomo - INFO - Saving flux vector to /content/drive/My Drive/Rotation 1/FBA_flux_file
INFO:pycomo:Saving flux vector to /content/drive/My Drive/Rotation 1/FBA_flux_file


,reaction_id,flux
Streptococcus_anginosus_1_2_62CV__2HMCOXT_Streptococcus_anginosus_1_2_62CV_c,Streptococcus_anginosus_1_2_62CV__2HMCOXT_Stre...,0.000000
Streptococcus_anginosus_1_2_62CV__2MBCOATA_Streptococcus_anginosus_1_2_62CV_c,Streptococcus_anginosus_1_2_62CV__2MBCOATA_Str...,0.000000
Streptococcus_anginosus_1_2_62CV__3HAD100_Streptococcus_anginosus_1_2_62CV_c,Streptococcus_anginosus_1_2_62CV__3HAD100_Stre...,0.000000
Streptococcus_anginosus_1_2_62CV__3HAD10M11_Streptococcus_anginosus_1_2_62CV_c,Streptococcus_anginosus_1_2_62CV__3HAD10M11_St...,0.000000
Streptococcus_anginosus_1_2_62CV__3HAD10M12_Streptococcus_anginosus_1_2_62CV_c,Streptococcus_anginosus_1_2_62CV__3HAD10M12_St...,0.000000
...,...,...
SK_Clostridium_bartlettii_DSM_16795_TF_no3_Clostridium_bartlettii_DSM_16795_e_ub,SK_Clostridium_bartlettii_DSM_16795_TF_no3_Clo...,1.428571
SK_Clostridium_bartlettii_DSM_16795_to_community_biomass_ub,SK_Clostridium_bartlettii_DSM_16795_to_communi...,1.346908
f_final,f_final,1.000000
abundance_reaction,abundance_reaction,57.164136
